This notebook reconstructs the orbits of fireball impactors listed in the <a href="https://cneos.jpl.nasa.gov/fireballs/">CNEOS Fireball Database</a> by performing backward numerical integrations using the REBOUND integrator. Given the impact location and velocity, we compute the orbital elements corresponding to each fireball’s orbit.

In [3]:
import spiceypy as spy
import pandas as pd
from Utils import *
import rebound as rb
import plotly.graph_objects as go
from scipy.optimize import newton
%load_ext autoreload 
%autoreload 2

In [4]:
#load the necessary spiceypy kernels

path = 'data/kernels/'
spy.furnsh([path + 'naif0012.tls', path + 'pck00010.tpc', path + 'earth_fixed.tf', path + 'earth_720101_230601.bpc', path + 'earth_latest_high_prec.bpc'])

In [14]:
#read the CNEOS fireball database

data_fireballs = pd.read_csv('data/cneos_fireballs.csv', comment='#')

## Chelyabinsk bolid backward integration example

Before performing the backward integration for all fireballs in the catalog, we begin by testing and illustrating the integration process using the well-known fireball associated with the Chelyabinsk impact event. By sorting the CNEOS fireball database in descending order by impact energy, we can easily identify the Chelyabinsk impactor, as this historic event currently holds the highest recorded impact energy.

In [11]:
data_fireballs.sort_values('Total Radiated Energy (J)', ascending=False).head(5)

,Peak Brightness Date/Time (UT),Latitude (deg.),Longitude (deg.),Altitude (km),Velocity (km/s),vx,vy,vz,Total Radiated Energy (J),Calculated Total Impact Energy (kt)
376,2013-02-15 03:20:33,54.8N,61.1E,23.3,18.6,12.8,-13.3,-2.4,3.750000e+14,440.0
178,2018-12-18 23:48:20,56.9N,172.4E,26.0,13.6,6.3,-3.0,-31.2,3.130000e+13,49.0
448,2010-12-25 23:24:00,38.0N,158.0E,26.0,18.1,18.0,-2.0,-4.0,2.000000e+13,33.0
494,2009-10-08 02:57:00,4.2S,120.6E,19.1,19.2,14.0,-16.0,-6.0,2.000000e+13,33.0
954,1994-02-01 22:38:09,2.7N,164.1E,NaN,NaN,NaN,NaN,NaN,1.820000e+13,30.0


In [13]:
#We located the Chelyabinsk bolid (chely) using the index id 37

id_chely = 376
chely = data_fireballs.loc[id_chely]
chely

Peak Brightness Date/Time (UT)         2013-02-15 03:20:33
Latitude (deg.)                                      54.8N
Longitude (deg.)                                     61.1E
Altitude (km)                                         23.3
Velocity (km/s)                                       18.6
vx                                                    12.8
vy                                                   -13.3
vz                                                    -2.4
Total Radiated Energy (J)                375000000000000.0
Calculated Total Impact Energy (kt)                  440.0
Name: 376, dtype: object

In [ ]:
#We change the Latitude and Longitude data format in the CNEOS database from string to a float value. 

def change_coord(x):
    if x[-1] == 'N' or x[-1] == 'E':
        new = float(x[:-1])
    elif x[-1] == 'S' or x[-1] == 'W':
        new = -float(x[:-1])
    return new    

date = chely['Peak Brightness Date/Time (UT)']

lon = change_coord(chely['Longitude (deg.)'])
lat = change_coord(chely['Latitude (deg.)'])
alt = chely['Altitude (km)']
vx = chely['vx']
vy = chely['vy']
vz = chely['vz']

print("Chelyabinsk impact data ")
print(f"Date of impact: {date}, Geographic coordinates: {lon, lat}")
print(f"Altitude: {alt}, velocity vector [{vx, vy, vy}]")

Chelyabinsk impact data 
Date of impact: 2013-02-15 03:20:33, Geographic coordinates: (61.1, 54.8)
Altitude: 23.3, velocity vector [(np.float64(12.8), np.float64(-13.3), np.float64(-13.3))]


In [ ]:
r = Geo2Rec(lon, lat, alt)  #en km
print("Rectangular coodinates: ", r, mag(r))
r_eclip = Geo2Eclip(lon, lat, alt, date=date, frame='ITRF93') #en km
print("Ecliptic Cooridnates: ", r_eclip, mag(r_eclip))

v = np.array([vx, vy, vz])  #en km/s

t_sideral = 86164.09053083288 
w_earth = 2 * np.pi / t_sideral 
omega = np.array([0,0,w_earth]) #rad/s

v_E = v + spy.vcrss(omega, r) #km/s
#v_E, -v, mag(v_E), np.arccos((v@r_irtf)/(np.linalg.norm(v)*np.linalg.norm(r_irtf)))*180/np.pi

et = spy.utc2et(date)
mx = spy.pxform('ITRF93', 'ECLIPJ2000', et)
v_eclip = spy.mxv(mx, v_E)

### Integration Time

We want to compute the pre-impact orbit of the bolids to find the semi major axis orbital element a to find the orbital period of each fireball. To do that, 

In [ ]:
fireballs = data_fireballs[["Peak Brightness Date/Time (UT)", "Latitude (deg.)", "Longitude (deg.)", "Altitude (km)", "vx", "vy", "vz"]].dropna().reset_index()
fireballs

In [ ]:
#we change the format of the geographical coordinates latitud and longitud to numerical values

fireballs['lat'] = fireballs["Latitude (deg.)"].apply(change_coord)
fireballs['lon'] = fireballs["Longitude (deg.)"].apply(change_coord)
fireballs[['lat','lon']]


In [7]:
rb.horizons.SSL_CONTEXT = 'unverified'
integration_times = []
nan_values = []
for i in range(len(fireballs)):
    print(f"--------- Integrating bolid {i} ------------")

    date = fireballs["Peak Brightness Date/Time (UT)"][i]
    lat = fireballs["lat"][i]
    lon = fireballs["lon"][i]
    alt = fireballs["Altitude (km)"][i]
    vx = fireballs['vx'][i]
    vy = fireballs['vy'][i]
    vz = fireballs['vz'][i]

    r_eclip = Geo2Eclip(lon, lat, alt, date=date, frame='ITRF93') #en km
    v_eclip = get_velocity_ecliptic(vx, vy, vz, lon, lat, alt, date=date)

    
    sim = rb.Simulation()
    sim.units = 'km', 's', 'kg'
    sim.integrator = "IAS15"
    sim.dt = -86400

    time = Time(date, format="iso")

    for body in ["Sun", "199", "299", "399", "499", "599", "699", "799", "899"]:
        sim.add(body, hash=f"{body}", date=f"JD{time.tdb.jd}")

    r_earth = np.array(sim.particles["399"].xyz)
    v_earth = np.array(sim.particles["399"].vxyz)

    r_asteroid = r_eclip + r_earth 
    v_asteroid = v_eclip + v_earth #así si es 

    asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                    vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2], hash="Asteroid")
    asteroid = sim.particles["Asteroid"]
    orbit = asteroid.orbit()

    a = orbit.a
    mu = sim.particles["Sun"].m*sim.G
    periodo_orb = 2*np.pi*np.sqrt(a**3/mu)

    t_end = 4*periodo_orb 

    print(f"-------------- time = ({t_end}) --------------")
    
    if np.isnan(t_end):
        nan_values.append(i)
    else:
        integration_times.append({'indice': i, 'integration_time': t_end})

--------- Integrating bolid 0 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Found: Jupiter (599) 
Searching NASA Horizons for '699'... 
Found: Saturn (699) 
Searching NASA Horizons for '799'... 
Found: Uranus (799) 
Searching NASA Horizons for '899'... 
Found: Neptune (899) 
-------------- time = (2475533080.4606795) --------------
--------- Integrating bolid 1 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Foun

C:\Users\aguju\AppData\Local\Temp\ipykernel_35124\2760486154.py:42: RuntimeWarning: invalid value encountered in sqrt
  periodo_orb = 2*np.pi*np.sqrt(a**3/mu)


Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Found: Jupiter (599) 
Searching NASA Horizons for '699'... 
Found: Saturn (699) 
Searching NASA Horizons for '799'... 
Found: Uranus (799) 
Searching NASA Horizons for '899'... 
Found: Neptune (899) 
-------------- time = (nan) --------------
--------- Integrating bolid 21 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Found: Jupiter (599) 
Searching NASA Horizons for '699'... 
Found: Saturn (699) 
Searching NASA Hor

In [9]:
integration_times

[{'indice': 0, 'integration_time': 2475533080.4606795},
 {'indice': 1, 'integration_time': 175802418.38179025},
 {'indice': 2, 'integration_time': 207472703.0005629},
 {'indice': 3, 'integration_time': 109389270.97236589},
 {'indice': 4, 'integration_time': 662624587.1550574},
 {'indice': 5, 'integration_time': 1610222542.42337},
 {'indice': 6, 'integration_time': 688221512.6128806},
 {'indice': 7, 'integration_time': 455402129.4497518},
 {'indice': 8, 'integration_time': 84755090.19181462},
 {'indice': 9, 'integration_time': 331976499.01356417},
 {'indice': 10, 'integration_time': 562810466.2521433},
 {'indice': 11, 'integration_time': 1224381049.4086673},
 {'indice': 12, 'integration_time': 73232680672.4767},
 {'indice': 13, 'integration_time': 816146269.9864191},
 {'indice': 14, 'integration_time': 171101309.23579615},
 {'indice': 15, 'integration_time': 397593698.84957737},
 {'indice': 16, 'integration_time': 681260992.5353541},
 {'indice': 17, 'integration_time': 88669785.21724984

In [ ]:
print(f"indices NaN values: {nan_values}")

indices NaN values: [19, 20, 23, 37, 49, 57, 58, 81, 86, 98, 114, 129, 136, 145, 152, 171, 183, 186, 189, 196, 204, 205, 210, 218, 246, 256, 261, 265, 279, 280, 283]


In [11]:
len(nan_values)

31

In [23]:
orbit_elements = np.zeros((len(integration_times), 6))

for i, time  in enumerate(integration_times):
    indice = time['indice']
    int_time = time['integration_time']

    print(f"--------- Integrating bolid {indice} ------------")
    date = fireballs["Peak Brightness Date/Time (UT)"][indice]
    lat = fireballs["lat"][indice]
    lon = fireballs["lon"][indice]
    alt = fireballs["Altitude (km)"][indice]
    vx = fireballs['vx'][indice]
    vy = fireballs['vy'][indice]
    vz = fireballs['vz'][indice]

    r_eclip = Geo2Eclip(lon, lat, alt, date=date, frame='ITRF93') #en km
    v_eclip = get_velocity_ecliptic(vx, vy, vz, lon, lat, alt, date=date)

    
    sim = rb.Simulation()
    sim.units = 'km', 's', 'kg'
    sim.integrator = "IAS15"
    sim.dt = -86400

    time = Time(date, format="iso")

    for body in ["Sun", "199", "299", "399", "499", "599", "699", "799", "899"]:
        sim.add(body, hash=f"{body}", date=f"JD{time.tdb.jd}")

    r_earth = np.array(sim.particles["399"].xyz)
    v_earth = np.array(sim.particles["399"].vxyz)

    r_asteroid = r_eclip + r_earth 
    v_asteroid = v_eclip + v_earth #así si es 

    asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                    vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2], hash="Asteroid")

    sim.integrate(-int_time)
    sim.move_to_hel()

    asteroid = sim.particles["Asteroid"]
    orbit = asteroid.orbit()

    orbit_elements[i] = [orbit.a, orbit.e, orbit.inc, orbit.Omega, orbit.omega, orbit.M]


--------- Integrating bolid 0 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Found: Jupiter (599) 
Searching NASA Horizons for '699'... 
Found: Saturn (699) 
Searching NASA Horizons for '799'... 
Found: Uranus (799) 
Searching NASA Horizons for '899'... 
Found: Neptune (899) 
--------- Integrating bolid 1 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Found: Jupiter (599) 
Searching NASA Horizons for '699'... 
Fo

### Save orbital elements 

In [27]:
fireballs_int = fireballs.drop(nan_values)

In [33]:
np.mod(orbit_elements[0][-1], 360)

6.022836564875549

In [28]:
fireballs_int['a (km)'] = orbit_elements[:,0]
fireballs_int['e'] = orbit_elements[:,1]
fireballs_int['i (rad)'] = orbit_elements[:,2]
fireballs_int['Omega (rad)'] = orbit_elements[:,3]
fireballs_int['omega (rad)'] = orbit_elements[:,4]
fireballs_int['M (rad)'] = orbit_elements[:,5]

orbital_elements_int = fireballs_int
orbital_elements_int.to_csv("datos/orbital_elements_integration.csv", index=False)

## Influence Sphere Background Integration 

In [ ]:
times = []

def SoiCross(time):

    sim.integrate(-time)
    r = np.linalg.norm(np.array(sim.particles[2].xyz) - np.array(sim.particles[1].xyz))

    return r - Rsoi

for i in range(len(fireballs)):
    print(f"--------- Integrating bolid {i} ------------")

    date = fireballs["Peak Brightness Date/Time (UT)"][i]
    lat = fireballs["lat"][i]
    lon = fireballs["lon"][i]
    alt = fireballs["Altitude (km)"][i]
    vx = fireballs['vx'][i]
    vy = fireballs['vy'][i]
    vz = fireballs['vz'][i]

    r_eclip = Geo2Eclip(lon, lat, alt, date=date, frame='ITRF93') #en km
    v_eclip = get_velocity_ecliptic(vx, vy, vz, lon, lat, alt, date=date)

    
    sim = rb.Simulation()
    sim.units = 'km', 's', 'kg'
    sim.integrator = "IAS15"
    sim.dt = -3600

    time = Time(date, format="iso")

    for i in ["Sun", "199", "299", "399", "499", "599", "699", "799", "899"]:
        sim.add(i, hash=f"{i}", date=f"JD{time.tdb.jd}")

    r_earth = np.array(sim.particles["399"].xyz)
    v_earth = np.array(sim.particles["399"].vxyz)

    r_asteroid = r_eclip + r_earth 
    v_asteroid = v_eclip + v_earth #así si es 

    asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                    vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2], hash="Asteroid")

    a = 1.496e8
    Rsoi = a * (sim.particles["399"].m/sim.particles["Sun"].m)**(2/5)

    t_soi = newton(SoiCross, 3600)

    times.append(t_soi)

--------- Integrating bolid 0 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '199'... 
Found: Mercury (199) 
Searching NASA Horizons for '299'... 
Found: Venus (299) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
Searching NASA Horizons for '499'... 
Found: Mars (499) 
Searching NASA Horizons for '599'... 
Found: Jupiter (599) 
Searching NASA Horizons for '699'... 
Found: Saturn (699) 
Searching NASA Horizons for '799'... 
Found: Uranus (799) 
Searching NASA Horizons for '899'... 
Found: Neptune (899) 


In [ ]:
print(f"Rango de horas cruce esfera de influencia: {np.min(times)/hora} - {np.max(times)/hora}")

Rango de horas cruce esfera de influencia: 5.69920221823576 - 115.72036993852151


In [ ]:
positions = []


for i in range(len(fireballs)):
    print(f"--------- Integrating bolid {i} ------------")
    date = fireballs["Peak Brightness Date/Time (UT)"][i]
    lat = fireballs["lat"][i]
    lon = fireballs["lon"][i]
    alt = fireballs["Altitude (km)"][i]
    vx = fireballs['vx'][i]
    vy = fireballs['vy'][i]
    vz = fireballs['vz'][i]

    r_eclip = Geo2Eclip(lon, lat, alt, date=date, frame='ITRF93') #en km
    v_eclip = get_velocity_ecliptic(vx, vy, vz, lon, lat, alt, date=date)

    
    sim = rb.Simulation()
    sim.units = 'km', 's', 'kg'
    sim.integrator = "IAS15"
    sim.dt = -3600

    time = Time(date, format="iso")

    for i in ["Sun", "199", "299", "399", "499", "599", "699", "799", "899"]:
        sim.add(i, hash=f"{i}", date=f"JD{time.tdb.jd}")

    r_earth = np.array(sim.particles["399"].xyz)
    v_earth = np.array(sim.particles["399"].vxyz)

    r_asteroid = r_eclip + r_earth 
    v_asteroid = v_eclip + v_earth #así si es 

    asteroid = sim.add(x=r_asteroid[0], y=r_asteroid[1], z=r_asteroid[2], 
                    vx=v_asteroid[0], vy=v_asteroid[1], vz=v_asteroid[2], hash="Asteroid")

    sim.integrate(-times[i])
    positions.append(np.array(sim.particles["Asteroid"].xyz) - np.array(sim.particles["399"].xyz))

positions = np.array(positions)

--------- Integrating bolid 0 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
--------- Integrating bolid 1 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
--------- Integrating bolid 2 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
--------- Integrating bolid 3 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
--------- Integrating bolid 4 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
--------- Integrating bolid 5 ------------
Searching NASA Horizons for 'Sun'... 
Found: Sun (10) 
Searching NASA Horizons for '399'... 
Found: Earth (399) 
--------- Integrating bolid 6 ------------
Searching NASA Horizo

In [ ]:
df = pd.DataFrame(columns=['x', 'y', 'z', 'r'])

df.x = positions[:,0]
df.y = positions[:,1]
df.z = positions[:,2]
r = [3e4 for j in range(len(positions[:,2]))]
df.r = r

In [ ]:
def ms(x, y, z, radius, resolution=20):
    """Return the coordinates for plotting a sphere centered at (x,y,z)"""
    u, v = np.mgrid[0:2*np.pi:resolution*2j, 0:np.pi:resolution*1j]
    X = radius * np.cos(u)*np.sin(v) + x
    Y = radius * np.sin(u)*np.sin(v) + y
    Z = radius * np.cos(v) + z
    return (X, Y, Z)

(x_earth_surface, y_earth_surface, z_earth_surface) = ms(0, 0, 0, Rsoi)
# Create a scatter plot
x_scatter = df.x  # Example scatter data
y_scatter = df.y
z_scatter = df.z

# Create 3D plot
fig = go.Figure()

# Solid sphere
fig.add_trace(go.Surface(x=x_earth_surface, y=y_earth_surface, z=z_earth_surface, colorscale='blues', opacity=0.5))

# Scatter plot
fig.add_trace(go.Scatter3d(x=x_scatter, y=y_scatter, z=z_scatter, mode='markers', marker=dict(size=3, color='red')))

fig.update_layout(scene=dict(aspectmode='data'), margin=dict(l=0, r=0, b=0, t=0))
fig.show()